In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import chardet
sns.set_palette('Set3')
%matplotlib inline

pd.options.display.max_columns = None
pd.options.display.max_rows = 20

import warnings
warnings.filterwarnings('ignore')

In [22]:
file_path = 'D:/dohyun/3grade/빅데이터/프로젝트/통합_위경도_클러스터.csv'

# 파일 경로 확인
if not os.path.exists(file_path):
    print("파일 경로가 존재하지 않습니다. 경로를 확인하세요.")
else:
    print("파일 경로가 유효합니다.")

# 파일 인코딩 확인
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read(100000))
    encoding = result['encoding']
    print(f"감지된 인코딩: {encoding}")

# CSV 파일 읽기
try:
    df_area = pd.read_csv(file_path, encoding=encoding)
    print(df_area.head())
except Exception as e:
    print(f"CSV 파일을 읽는 중 오류가 발생했습니다: {e}")

파일 경로가 유효합니다.
감지된 인코딩: UTF-8-SIG
  상권_구분_코드 상권_구분_코드_명  상권배후지코드 상권배후지_코드_명    행정동_코드 행정동_코드_명  총_직장_인구_수   
0        A       골목상권  3110001     이북5도청사  11110560      평창동    3168.80  \
1        A       골목상권  3110002    독립문역 1번  11110570      무악동     626.60   
2        A       골목상권  3110003    세검정초등학교  11110550      부암동    3108.85   
3        A       골목상권  3110004     대신고등학교  11110570      무악동     412.60   
4        A       골목상권  3110005        세검정  11110550      부암동    2076.70   

   남성_직장_인구_수  여성_직장_인구_수  연령대_10_직장_인구_수  연령대_20_직장_인구_수  연령대_30_직장_인구_수   
0     2230.30      938.50            9.55          569.65          738.90  \
1      179.95      446.65            6.65          167.20          171.05   
2     1624.25     1484.60           11.55          371.95          820.45   
3      234.85      177.75            5.75           56.85           97.35   
4     1059.25     1017.45            7.75          294.65          552.05   

   연령대_40_직장_인구_수  연령대_50_직장_인구_수  연령대_60_이상_직장_인구_수 

In [23]:
print(df_area[df_area['cluster'] == 23]['상권배후지_코드_명'])

22    서울대병원
Name: 상권배후지_코드_명, dtype: object


In [24]:
from folium.plugins import MarkerCluster
import folium
import pandas as pd

# NaN 및 소수점 처리
df_area['당월_매출_금액'] = pd.to_numeric(df_area['당월_매출_금액'], errors='coerce').fillna(0).astype(int)

# 지도 중심 계산
mean_lat = df_area['lat'].mean()
mean_lon = df_area['lon'].mean()

# Folium 지도 생성
map_with_clusters = folium.Map(location=[mean_lat, mean_lon], zoom_start=11)

# MarkerCluster 추가
marker_cluster = MarkerCluster().add_to(map_with_clusters)

# 데이터 클러스터에 추가
for index, row in df_area.iterrows():
    popup_content = (
        f"<b>상권배후지:</b> {row['상권배후지_코드_명']}<br>"
        f"<b>당월 매출 금액:</b> {row['당월_매출_금액']:,}원"
    )
    popup = folium.Popup(popup_content, max_width=300)  # 팝업 폭 설정
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=popup
    ).add_to(marker_cluster)

# Vworld 공용 타일 설정
vworld_tiles = "http://xdworld.vworld.kr:8080/2d/Base/202002/{z}/{x}/{y}.png"
attr = "Vworld (Public)"

folium.TileLayer(
    tiles=vworld_tiles,
    attr=attr,
    overlay=False,
    control=True
).add_to(map_with_clusters)

# 지도 HTML로 저장
output_file = 'D:/dohyun/3grade/빅데이터/프로젝트/map_with_clusters.html'
map_with_clusters.save(output_file)
print(f"지도가 '{output_file}'에 저장되었습니다.")


지도가 'D:/dohyun/3grade/빅데이터/프로젝트/map_with_clusters.html'에 저장되었습니다.
